## Скрипт для парсинга диалогов

Необходимо написать скрипт для парсинга диалогов из файла
test_data.csv.

Главные задачи, которые должен выполнять скрипт:
- Извлекать реплики с приветствием – где менеджер поздоровался.
- Извлекать реплики, где менеджер представил себя.
- Извлекать имя менеджера.
- Извлекать название компании.
- Извлекать реплики, где менеджер попрощался.
- Проверять требование к менеджеру: «В каждом диалоге
обязательно необходимо поздороваться и попрощаться с
клиентом»

## Решение

Для решения задачи используем Yargy parser. Подготовим правила для парсинга и затем последовательно применим их к диалогам.

In [1]:
#импорт библиотек
from yargy import rule, Parser, or_
from ipymarkup import show_span_box_markup as show_markup
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from yargy.predicates import gram
from yargy.relations import gnc_relation

from datetime import datetime
import os

import pandas as pd

pd.set_option('display.max_colwidth', None)

In [2]:
#читаем данные
df = pd.read_csv('msg/test_data.csv')

In [3]:
#первые пять строк
df.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
4,0,4,client,Ага


Перенесем в отдельный DataFrame все реплики менеджера и dlg_id для удобства исследования.

In [4]:
#создаем пустой список
dialog = []

#Заполняем список репликами менеджера с разбивкой по dlg_id
for dlg_id in df.dlg_id.unique():
    dialog.append([dlg_id, '. '.join(x for x in df.loc[(df['dlg_id'] == dlg_id) & (df['role'] == 'manager')]['text'])])

#из полученного списка делаем DataFrame с колонками 'dlg_id', 'text'
manager = pd.DataFrame(dialog, columns = ['dlg_id', 'text'])

In [5]:
'''
Правило для имени менеджера
1. Объявляем факт Name
2. Создаем грамему с именами
3. Создаем граммему с фамилиями
4. Создаем словарь возможных вводных слов
5. Создаем правило NAME_RULE с различнми вариантами последовательносте имя-фамилия
6. Создаем правило NAME
'''
Name = fact(
    'Name',
    ['introduce', 'first', 'last']
)

# согласование по gender, number и case (падежу, числу и роду)
gnc = gnc_relation() 

#имя
FIRST = gram('Name').interpretation(
    Name.first.inflected().custom(lambda x: x.title())
).match(gnc)

#фамилия
LAST = gram('Surn').interpretation(
    Name.last.inflected().custom(lambda x: x.title())
).match(gnc)

#вводные слова
INTRODUCE = morph_pipeline({
    'зовут',
    'это',
    'меня',
    'меня зовут',
}).interpretation(
    Name.introduce.normalized()
)

#правила написания имени и фамилии (ИФ/ФИ/И)
NAME_RULE = or_(
    rule(
        FIRST,
        LAST
    ),
    rule(
        LAST,
        FIRST
    )
    ,
    rule(
        FIRST
    )
)
#правило для представления менеджера
NAME = rule(
    INTRODUCE,
    NAME_RULE
).interpretation(
    Name
)

In [6]:
'''
Правило для прощания
1. Объявляем факт Goodbye
2. Создаем словарь прощаний
3. Создаем правило GOODBYE_LOC
'''

Goodbye  = fact(
    'Goodbye',
    ['word']
)

GOODBYE_WORD = morph_pipeline([
    'до свидания',
    'хорошего вечера',
    'хорошего дня',
    'всего доброго'
]).interpretation(
    Goodbye.word.normalized().custom(lambda x: x.capitalize())
)

GOODBYE_LOC = rule(
    GOODBYE_WORD
).interpretation(
    Goodbye
)

In [7]:
'''
Правило для названия компании
1. Объявляем факт Company
2. Создаем словарь словоформ для слова компания
3. Создаем словарь названий компаний
4. Создаем правило COMPANY_LOC
'''

Company  = fact(
    'Company',
    ['form','name']
)

COMPANY_FORM = morph_pipeline([
    'компания',
    'фирма'
]).interpretation(
    Company.form.normalized()
)

COMPANY_NAME = morph_pipeline([
    'диджитал бизнес',
    'китобизнес'
]).interpretation(
    Company.name.normalized().custom(lambda x: x.title())
)

COMPANY_LOC = rule(
    COMPANY_FORM.optional(),
    COMPANY_NAME
).interpretation(
    Company
)

In [8]:
'''
Правило для приветствия
1. Объявляем факт Greetings
2. Создаем словарь приветствий
3. Создаем правило GRITINGS_LOC
'''
Greetings  = fact(
    'Greetings',
    ['word']
)

GREETINGS_WORD = morph_pipeline([
    'здравствуйте',
    'добрый день',
    'доброе утро',
    'добрый вечер',
    'доброй ночи',
    'привет',
    'здравствуй',
    'приветствую вас'
]).interpretation(
    Greetings.word.normalized().custom(lambda x: x.capitalize())
)

GREETINGS_LOC = rule(
    GREETINGS_WORD
).interpretation(
    Greetings
)

In [9]:
#Запускаем парсинг по очереди применяя каждое правило
#ищем приветсвие
parser = Parser(GREETINGS_LOC)
manager['greeting'] = manager.text.apply(lambda x: ', '.join(
    [match.fact.word for match in parser.findall(x)]))
#ищем как представился менеджер
parser = Parser(NAME)
manager['manager_name'] = manager.text.apply(lambda x: ', '.join(
    [match.fact.first+(' ' + match.fact.last if match.fact.last else '')  for match in parser.findall(x)]))
#ищем название компании
parser = Parser(COMPANY_LOC)
manager['company'] = manager.text.apply(lambda x: ', '.join(
    [(match.fact.form+' ' if match.fact.form else '') + match.fact.name for match in parser.findall(x)]))
#ищем прощание
parser = Parser(GOODBYE_LOC)
manager['goodbye'] = manager.text.apply(lambda x: ', '.join(
    [match.fact.word for match in parser.findall(x)]))

In [10]:
#сами реплики менеджера не выводим по соображениям конфиденциальности
manager[['dlg_id', 'greeting', 'manager_name', 'company', 'goodbye']]

,dlg_id,greeting,manager_name,company,goodbye
0,0,Здравствуйте,Ангелина,компания Диджитал Бизнес,До свидания
1,1,Здравствуйте,Ангелина,компания Диджитал Бизнес,До свидания
2,2,Здравствуйте,Ангелина,"компания Диджитал Бизнес, Диджитал Бизнес",
3,3,Добрый день,Максим,компания Китобизнес,Всего доброго
4,4,,,,До свидания
5,5,,Анастасия,,"До свидания, Хорошего вечера"


In [11]:
manager['is_greeting'] = (manager['greeting']!='')*1
manager['is_goodbye'] = (manager['goodbye']!='')*1
manager['is_right'] = (manager['goodbye']!='')*1 & (manager['greeting']!='')*1

In [12]:
#сами реплики менеджера не выводим по соображениям конфиденциальности
manager[['dlg_id', 'greeting', 'is_greeting', 'manager_name', 'company', 'goodbye', 'is_goodbye', 'is_right']]

,dlg_id,greeting,is_greeting,manager_name,company,goodbye,is_goodbye,is_right
0,0,Здравствуйте,1,Ангелина,компания Диджитал Бизнес,До свидания,1,1
1,1,Здравствуйте,1,Ангелина,компания Диджитал Бизнес,До свидания,1,1
2,2,Здравствуйте,1,Ангелина,"компания Диджитал Бизнес, Диджитал Бизнес",,0,0
3,3,Добрый день,1,Максим,компания Китобизнес,Всего доброго,1,1
4,4,,0,,,До свидания,1,0
5,5,,0,Анастасия,,"До свидания, Хорошего вечера",1,0


In [13]:
#Выгрузка отчета. В название отчета добавим текщее время в выгрузим в подпапку msg
file_path = os.path.abspath(os.curdir)
path = os.path.join(file_path, "msg", 'report_'+str(datetime.now().strftime("%d_%m_%Y_%H_%M_%S"))+'.csv')
manager[['dlg_id', 'greeting', 'is_greeting', 'manager_name', 'company', 'goodbye', 'is_goodbye', 'is_right']].to_csv(path, sep = ';', encoding='utf-8-sig')

### Вывод

Диалоги парсятся с помощью библиотеки Yargy parser, которая предоставляет простой и удобный способ работы с текстами.
Таблица с результатами парсинга подготовлена:
- greeting столбец приветсвий
- is_greeting столбец проверки наличия приветсвия
- manager_name имя менеджера
- company упоминания компании внутри диалога
- goodbye столбец с репликами прощания
- is_goodbye столбец проверки наличия прощания
- is_right столбец проверки наличия и приветсвия и прощания

У каждого диалога для идентификации сохранен номер из изначального файла для возможности сопоставления диалогов.

У сущностей приветствие, название компании, прощание сопоставление идет по словарю, который лекго может быть отредактирован, если у компании есть определенный стандарт приветсвий и прощаний.
Кроме  того, если у компании есть определенный крипт продаж с ключевыми моментами, которые менеджер должен проговорить, то поиск таких сущностей тоже можно добавить.